# ACR Inherent Risk Analysis Tool

In [1]:
# load geemap and other packages
import os
import ee
import geemap
from glob import glob
import pandas as pd
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
import math

In [2]:
# Trigger the authentication flow
# need to do it when it is first time to run this notebook
# will be directed for permission confirmation 
# and The authorization workflow will generate a code, 
# which you should paste in the box below.
ee.Authenticate()

Enter verification code:  4/1AVMBsJgwT_lJjB-VNt7U-jhtfP2hbjqVGorBkrSbXew6TcY_osv5nZ-fYSI



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [14]:
# direct to project area shapefile
# replace file location and name as you want below
project_area_path = r'Projects/ACR/638/638_wgs84_p_1ha.shp'

Projects/ACR/638/638_wgs84_p_1ha.shp


Possible issue encountered when converting Shape #0 to GeoJSON: Shapefile format requires that all polygon interior holes be contained by an exterior ring, but the Shape contained interior holes (defined by counter-clockwise orientation in the shapefile format) that were orphaned, i.e. not contained by any exterior rings. The rings were still included but were encoded as GeoJSON exterior rings instead of holes.


In [11]:
# upload it to google earth engine
# calculate its project area in ha and acre
pa = geemap.shp_to_ee(project_area_path)
pa_area = round(pa.geometry().area(1).getInfo()/10000, 2)
print(pa_area, ' ha')
print(pa_area * 2.47, 'acre')

Projects/ACR/378/378_wgs84_p_1ha.shp
787.56  ha
1945.2732 acre


In [6]:
# Display project area in google earth engine
Map = geemap.Map()
Map.centerObject(pa, 12)
Map.addLayer(pa, {'color': 'red'}, 'Project Area')
Map

### Global Forest Change Dataset for piror logging check

In [7]:
# load avaliable dataset for analysis
# Global Forest Watch or others for piror logging check
gfc = ee.Image(r'UMD/hansen/global_forest_change_2023_v1_11')

# tree cover 2000
gfc_treecover_2000 = gfc.select(['treecover2000'])

# tree loss 
gfc_loss_image = gfc.select(['loss'])

# tree loss year (range from 1 to 21), base year is 2000.
gfc_loss_year = gfc.select(['lossyear'])

In [24]:
# Display project area and GFC dataset
Map = geemap.Map()

# check the tree loss before project start date
Map.addLayer(gfc_loss_year.updateMask(gfc_loss_year.lte(20)),
            {'min': 0,
            'max': 20,
            'palette': ['yellow', 'red']},
            'Tree loss before 2020')

Map.centerObject(pa, 12)
Map.addLayer(pa, {'color': 'red'}, 'Project Area')
Map

In [9]:
# a function to get annual deforestation (in ha) within project area using GFC dataset
def get_annual_loss(loss_image, loss_year, region):
    loss_area = loss_image.multiply(ee.Image.pixelArea())
    loss_area_year = loss_area.addBands(loss_year).reduceRegion(
        **{'reducer': ee.Reducer.sum().group(**{'groupField': 1}),
           'geometry': region,
           'scale': 30,
           'bestEffort': True,
           'maxPixels': 1e9})
    loss_area_year = loss_area_year.getInfo()['groups']
    years, annual_deforestation = [], []
    for g in loss_area_year:
        years.append(g['group'])
        annual_deforestation.append(g['sum']/10000)    
    return years, annual_deforestation

In [10]:
pa_years_def_gfc, pa_annual_deforestation_gfc = get_annual_loss(gfc_loss_image, 
                                                                gfc_loss_year, 
                                                                pa)

In [11]:
df_gfc = pd.DataFrame({'Def_PA': pa_annual_deforestation_gfc}, index=[d+2000 for d in pa_years_def_gfc])

In [12]:
df_gfc['Def_PA_rate (%)'] = df_gfc['Def_PA'] * 100 / pa_area
df_gfc

,Def_PA,Def_PA_rate (%)
2001,0.230372,0.006998
2002,0.250149,0.007599
2004,1.063410,0.032303
2006,1.463215,0.044447
2007,0.371506,0.011285
2008,0.745012,0.022631
2009,0.175404,0.005328
2010,0.840953,0.025545
2011,0.270145,0.008206
2014,0.962038,0.029223


### Digital elevation model (DEM) for forever unsuitable harvest analysis

In [38]:
# Display add DEM on project area
Map = geemap.Map()

dataset = ee.Image('USGS/3DEP/10m').clip(pa)
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)

Map.addLayer(elevation, {'min': 0, 'max': 3000, 'palette': [
    '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
    'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
    '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
  ],
}, 'project elevation')
Map.addLayer(slope, {'min': 0, 'max': 60}, 'project slope')

Map.centerObject(pa, 12)
Map.addLayer(pa, {'color': 'red'}, 'Project Area')
Map

In [19]:
# get histogram of slope distribution with project area
histogram = slope.reduceRegion(
    reducer=ee.Reducer.fixedHistogram(0, 90, 18),
    geometry=pa,
    scale=10,
    maxPixels=1e9
)

In [20]:
hist_data = histogram.get('slope').getInfo()

In [22]:
df = pd.DataFrame(hist_data, columns=['Slope [v, v+5)', 'Frequency'])
df['Percent (%)'] = (df['Frequency'] / df['Frequency'].sum()) * 100
df['CumulativePercent (%)'] = df['Percent (%)'].cumsum()
df

,"Slope [v, v+5)",Frequency,Percent (%),CumulativePercent (%)
0,0,75908.023529,8.946470,8.946470
1,5,151235.152941,17.824476,26.770945
2,10,150912.870588,17.786492,44.557437
3,15,128773.109804,15.177114,59.734551
4,20,105703.976471,12.458201,72.192752
5,25,85640.207843,10.093499,82.286251
6,30,69164.047059,8.151629,90.437880
7,35,47279.117647,5.572286,96.010166
8,40,22973.933333,2.707693,98.717859
9,45,7793.992157,0.918595,99.636453


In [32]:
# you can change slope to 30 or lower
print(f"{100-df.loc[df['Slope [v, v+5)'] == 25]['CumulativePercent (%)'].item()} %")

17.713748941241974 %


### ESA World Cover (10 m) for land cover in project area

In [18]:
Map = geemap.Map()
dataset = ee.ImageCollection('ESA/WorldCover/v100').first()
visualization = {
  'bands': ['Map'],
}

Map.addLayer(dataset.clip(pa), visualization, 'Landcover')
Map

In [28]:
map_class_table = {'Tree Cover': 10,
                   'Shrubland': 20,
                   'Grassland': 30,
                   'Cropland': 40,
                   'Built-up': 50,
                   'Bare/sparse vegetation': 60,
                   'Snow and ice': 70,
                   'Permanent water bodies': 80,
                   'Herbaceous wetland': 90,
                   'Mangroves': 95,
                   'Moss and lichen': 100}

landcover_counts = dataset.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                                        geometry=pa, scale=10, maxPixels=1e9)

class_counts = landcover_counts.get('Map').getInfo()

df = pd.DataFrame(list(class_counts.items()), columns=['LandCoverClass', 'PixelCount'])
df['Percentage'] = (df['PixelCount'] / df['PixelCount'].sum()) * 100
df

### Near-water forest areas

In [39]:
water_bodies = dataset.eq(80)
distance_to_water = water_bodies.Not().cumulativeCost(
    source=water_bodies, 
    maxDistance=30
)
buffered_water = distance_to_water.lte(20)

In [40]:
forest = dataset.eq(10) # Binary forest mask (1 = forest, 0 = non-forest)

# Intersect the forest raster with the buffered water mask
forest_near_water = forest.updateMask(buffered_water)

# Calculate the total forest coverage within 30 meters of water bodies
forest_area = forest_near_water.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=pa,  # Use the extent of the forest image
    scale=10,  # Match the dataset resolution
    maxPixels=1e9
)

In [41]:
# Get the result
total_forest_area = forest_area.get('Map').getInfo()  # 'constant' band holds the binary mask

In [42]:
print(f"Total forest area within 20 meters of water: {total_forest_area / 1e4:.2f} hectares, {total_forest_area / 1e4 / pa_area:.4f}")

Total forest area within 20 meters of water: 51.32 hectares, 0.0099


In [38]:
print(f"Total forest area within 30 meters of water: {total_forest_area / 1e4:.2f} hectares, {total_forest_area / 1e4 / pa_area:.4f}")

Total forest area within 30 meters of water: 91.43 hectares, 0.0176


### imminent harvest analysis using biomass map and tree height map 

In [12]:
# biomass datase ESA 
agb = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
agb_2010 = agb.filter(ee.Filter.date('2010-01-01','2011-01-01')).first().select(['AGB'])
agb_2017 = agb.filter(ee.Filter.date('2017-01-01','2018-01-01')).first().select(['AGB'])
agb_2018 = agb.filter(ee.Filter.date('2018-01-01','2019-01-01')).first().select(['AGB'])
agb_2019 = agb.filter(ee.Filter.date('2019-01-01','2020-01-01')).first().select(['AGB'])
agb_2020 = agb.filter(ee.Filter.date('2020-01-01','2021-01-01')).first().select(['AGB'])

In [10]:
def get_histogram_biomass(agb, pa, year, crs):
    agb = agb.updateMask(agb.mask())
    # Calculate the histogram using the fixed range and bin size
    hist = agb.reduceRegion(
        reducer=ee.Reducer.fixedHistogram(0, 400, 40),
        geometry=pa, 
        scale=100, 
        maxPixels=1e9,
        crs=f'EPSG:{crs}'
    )
    
    # Extract histogram data
    hist_data = hist.get('AGB').getInfo()
    # Extract bin edges and counts
    bin_edges = [h[0] for h in hist_data]
    frequencies = [h[1] for h in hist_data]
    # print(sum(frequencies))
    # Convert the bin edges to biomass intervals
    intervals = [(bin_edges[i], bin_edges[i] + 10) for i in range(len(bin_edges))]

    # Calculate the total area per biomass interval
    hectares = [frequency * 1 for frequency in frequencies]
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Biomass Interval (Mg/ha)': [f'{interval[0]} - {interval[1]}' for interval in intervals],
        f'ESA_{year}_Hectares': hectares
    })
    return df

In [20]:
# Display with biomass map
Map = geemap.Map()
palette = ["#C6ECAE","#A1D490","#7CB970","#57A751","#348E32", "#267A29","#176520","#0C4E15","#07320D","#031807"]
Map.addLayer(agb_2020.clip(pa), {'min': 0, 'max': 450, 'palette': palette}, 'agb_2020')
Map

In [ ]:
histogram = agb_2010.reduceRegion(reducer=ee.Reducer.fixedHistogram(0, 450, 45), 
                                      geometry=pa, scale=100, crs=f'EPSG:{epsg_number}', maxPixels=1e9)
hist_data = histogram.get('AGB').getInfo()
df = pd.DataFrame(hist_data, columns=['Biomass (Mg/ha) [v, v+10)', 'Frequency'])
df['Percent (%)'] = (df['Frequency'] / df['Frequency'].sum()) * 100
df['CumulativeP (%)'] = df['Percent (%)'].cumsum()
print(df)
print(f"Areas with biomass > 270 Mg/ha: {round(100-df.loc[df['Biomass (Mg/ha) [v, v+10)'] == 230]['CumulativeP (%)'].item(), 2)} %")

In [3]:
# tree height dataset 
canopy_ht_meta = ee.ImageCollection("projects/meta-forest-monitoring-okw37/assets/CanopyHeight").mosaic()
canopy_ht_meta = canopy_ht_meta.updateMask(canopy_ht_meta.gte(1))
canopy_ht_eth = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")

In [4]:
def get_height_hist(canopy_height, pa, scale, crs):
    # Calculate the histogram using the fixed range and bin size
    hist = canopy_height.reduceRegion(
        reducer=ee.Reducer.fixedHistogram(0, 50, 10),
        geometry=pa, 
        scale=scale, 
        maxPixels=1e9,
        crs=f'EPSG:{crs}'
    )
    # Extract histogram data
    if scale == 1:
        hist_data = hist.get('cover_code').getInfo()
    else:
        hist_data = hist.get('b1').getInfo()
    
    # Extract bin edges and counts
    bin_edges = [h[0] for h in hist_data]
    frequencies = [h[1] for h in hist_data]
    # print(sum(frequencies))
    # Convert the bin edges to biomass intervals
    intervals = [(bin_edges[i], bin_edges[i] + 5) for i in range(len(bin_edges))]
    
    s = scale * scale / 10000
    # Calculate the total area per biomass interval
    hectares = [frequency * s for frequency in frequencies]
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Canopy_Height(m)': [f'{interval[0]} - {interval[1]}' for interval in intervals],
        f'Area': hectares
    })
    return df

In [ ]:
# tree height
histogram = canopy_height.reduceRegion(reducer=ee.Reducer.fixedHistogram(0, 50, 10), 
                                           geometry=pa, crs=f'EPSG:{epsg_number}', scale=10, maxPixels=1e9)
hist_data = histogram.get('b1').getInfo()
df = pd.DataFrame(hist_data, columns=['Tree Height (m) [v, v+5)', 'Frequency'])
df['Percent (%)'] = (df['Frequency'] / df['Frequency'].sum()) * 100
df['CumulativeP (%)'] = df['Percent (%)'].cumsum()
print(df)
print(f"Areas with tree height > 30 m: {round(100-df.loc[df['Tree Height (m) [v, v+5)'] == 25]['CumulativeP (%)'].item(), 2)} %")

In [22]:
Map = geemap.Map()
canopy_vis = {"min":0,"max":50,
              "palette":["#010005","#150b37","#3b0964","#61136e","#85216b","#a92e5e","#cc4248","#e75e2e","#f78410","#fcae12","#f5db4c","#fcffa4"]}
# sd_vis = {"min":0,"max":15,
#           "palette":["#0d0406","#241628","#36274d","#403a76","#3d5296","#366da0","#3488a6","#36a2ab","#44bcad","#6dd3ad","#aee3c0","#def5e5"]},

canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")
# standard_deviation = ee.Image("users/nlang/ETH_GlobalCanopyHeightSD_2020_10m_v1")

Map.addLayer(canopy_height.clip(pa), canopy_vis, 'Canopy top height 2020')
Map

### WDPA (polygon) dataset for calculating protection areas

In [37]:
Map = geemap.Map()
dataset = ee.FeatureCollection('WCMC/WDPA/current/polygons')
visParams = {
  'palette': ['2ed033', '5aff05', '67b9ff', '5844ff', '0a7618', '2c05ff'],
  'min': 0.0,
  'max': 1550000.0,
  'opacity': 0.8,
}
image = ee.Image().float().paint(dataset, 'REP_AREA')
Map.addLayer(image, visParams, 'WCMC/WDPA/current/polygons')
Map

In [36]:
project_geometry = pa.geometry()

# Filter WDPA polygons that intersect the project area
wdpa_filtered = dataset.filterBounds(project_geometry)

# Calculate intersections
overlap = wdpa_filtered.map(lambda feature: feature.set(
    'ha', feature.intersection(project_geometry).area(1).divide(1e4)
))

total_overlap_area = overlap.aggregate_sum('ha').getInfo()
print(total_overlap_area)